In [36]:
''' This notebook contains code for note processing.

Contributors:
    Sam Sui
'''

# Standard libraries
import re
import requests

# Third-party libraries
import numpy as np
import pandas as pd

In [28]:
url = 'https://patchalysis.com/api/patches'
patches = requests.get(url).json()
patches = [patch for patch in patches['patches']] # List of patch versions (ex. V14_8)
patches

['V0_8_21_110',
 'V0_8_22_115',
 'V0_9_22_15',
 'V0_9_22_16',
 'V0_9_22_18',
 'V0_9_22_4',
 'V0_9_22_7',
 'V0_9_22_9',
 'V0_9_25_21',
 'V0_9_25_24',
 'V0_9_25_34',
 'V10_1',
 'V10_10',
 'V10_11',
 'V10_12',
 'V10_13',
 'V10_14',
 'V10_15',
 'V10_16',
 'V10_17',
 'V10_18',
 'V10_19',
 'V10_2',
 'V10_20',
 'V10_21',
 'V10_22',
 'V10_23',
 'V10_24',
 'V10_25',
 'V10_3',
 'V10_4',
 'V10_5',
 'V10_6',
 'V10_7',
 'V10_8',
 'V10_9',
 'V11_1',
 'V11_10',
 'V11_11',
 'V11_12',
 'V11_13',
 'V11_14',
 'V11_15',
 'V11_16',
 'V11_17',
 'V11_18',
 'V11_19',
 'V11_2',
 'V11_20',
 'V11_21',
 'V11_22',
 'V11_23',
 'V11_24',
 'V11_3',
 'V11_4',
 'V11_5',
 'V11_6',
 'V11_7',
 'V11_8',
 'V11_9',
 'V12_1',
 'V12_10',
 'V12_11',
 'V12_12',
 'V12_13',
 'V12_14',
 'V12_15',
 'V12_16',
 'V12_17',
 'V12_18',
 'V12_19',
 'V12_2',
 'V12_20',
 'V12_21',
 'V12_22',
 'V12_23',
 'V12_3',
 'V12_4',
 'V12_5',
 'V12_6',
 'V12_7',
 'V12_8',
 'V12_9',
 'V13_1',
 'V13_10',
 'V13_11',
 'V13_12',
 'V13_13',
 'V13_14',
 'V13_

In [32]:
''' Create a DataFrame to store patch notes, champion, win rate, and patch version
'''

patch_notes_base_url = 'https://patchalysis.com/api/patch/notes/'
patch_stats_base_url = 'https://patchalysis.com/api/patch/stats/'

df = pd.DataFrame(columns=['version', 'champion', 'note', 'win_rate'])

for patch_version in patches:
    notes_url = patch_notes_base_url + patch_version
    stats_url = patch_stats_base_url + patch_version

    notes_data = requests.get(notes_url).json()
    stats_data = requests.get(stats_url).json()

    if notes_data is None or stats_data is None: continue

    # Extract champion win rates
    win_rates = {}
    for champion in stats_data['champions']:
        champion_name = champion['champion']
        win_rate = champion['win-rate']

        # if win_rate is None: continue # Skip champions with no win rate data (null)
        win_rate = float(win_rate.replace('%', '')) if win_rate is not None else np.nan # Convert win rate to float
        win_rates[champion_name] = win_rate

    # Extract champion notes
    for champion in notes_data['champions']:
        champion_name = champion['champion']
        match = re.match(r"^([^\(-]+)", champion_name)
        if match:
            champion_name = match.group(1).strip()
        champion_updates = champion['updates']

        if champion_name not in win_rates:
            win_rate = np.nan
        else:
            win_rate = win_rates[champion_name]

        for update in champion_updates:
            for aspect in update:
                note = update[aspect]
                if note is None: continue
                df.loc[len(df)] = [patch_version, champion_name, note, win_rate]

In [33]:
df.head()

,version,champion,note,win_rate
0,V10_1,Aphelios,Movement speed reduced to 325 from 330. Base h...,54.0
1,V10_1,Aphelios,Infernum splash damage reduced to 75% from 100%.,54.0
2,V10_1,Aurelion Sol,Base damage reduced to 12 − 120 (based on leve...,13.0
3,V10_1,Azir,Base damage increased to 175 / 325 / 475 from ...,46.0
4,V10_1,Corki,Base damage increased to 90 / 125 / 160 from 9...,46.0


In [35]:
# df.to_csv('patches.csv', index=False, na_rep='NaN')